In [2]:
import pandas as pd
df = pd.read_csv("kospi200_raw.csv", index_col = 0)
df.columns = ['date','close','open','high','low','volume','name']
df['date'] = pd.to_datetime(df['date'])

In [10]:
df.head(1)

,date,close,open,high,low,volume,name
60,2018-01-02,210500.0,212500.0,212500.0,208500.0,43076.0,BGF리테일


In [11]:
import chardet

In [30]:
filename = "C:\\Users\\mi-bi\\OneDrive\\바탕 화면\\Kis Pjt\\Kis_Api_test\\4_SRC\\kospi200_raw.csv"
with open(filename, 'rb') as f:
    result = chardet.detect(f.readline())  # or read() if the file is small.
    print(result['encoding'])

utf-8


### Cerebro  
Backtrader의 주춧돌 역할을 하는 메인클래스 

In [13]:
import backtrader as bt
cerebro = bt.Cerebro()

모든 입력을 모아주는 센터

<가능한 입력>  
DataFeeds: 과거 주가 데이터  
Strategies: 매수매도 전략  
Writer: 문서화, Analyzer: 분석, Observer: 감시  
Broker: 거래소  
Notificatio: 알림  
과거 주가 데이터, 혹은 실시간 데이터에 대해 Backtesting을 수행합니다.  
결과를 반환합니다.  
그래프 그리기 등의 외부 라이브러리에 대한 접근을 제공합니다.  

### Data Feeds (GenericCSVData)

Cerebro 데이터 입력방법 야후 파이낸스, Pandas (우리 PJT는 dataframe)  
datetime, time, high, row, open, colse, volum 등 데이터 위치 양식 명세 

In [68]:
data = bt.feeds.GenericCSVData(
            dataname="kospi200_raw.csv",

            fromdate=datetime.datetime(2018, 1, 2),
            todate=datetime.datetime(2022, 12, 29),

            nullvalue=0.0,

            dtformat=('%Y-%m-%d'),
            #tmformat=('%I-%p'),

            datetime=0,
            time=-1,
            high=3,
            low=4,
            open=2,
            close=1,
            volume=5,
            openinterest=-1,

                        headers = True,
                        separator = ','
        )

cerebro.adddata(data)

### Strategy

cerebro에 매수매도 전략 전달 

<생애주기>  
* Initialize (__init__) : 전략에서 이용하는 지표들(indicators) 정의  
* Birth (start) : World(cerobro)에게 시작을 알림.  
* Childhood (prenext) : Window Size가 아직 차지 않은 초기 단계.  
* Adulthood (next) : Window Size만큼의 데이터를 축적하여 본격적인 예측을 할 수 있는 단계.  
* Reproduction : Parameter Optimizing 등을 위해 전략을 복사할 때 이용.  
* Death (stop) : 종료  

In [69]:
class MyStrategy(bt.Strategy):

    def __init__(self):
        self.sma = btind.SimpleMovingAverage(period=15)

    def next(self):
        if self.sma > self.data.close:
            # Do something
            pass

        elif self.sma < self.data.close:
            # Do something else
            pass

__init__ 과 next 로 구성됨 next에 상세 전략 필요함녀 헬퍼함수 추가    
start, prenext, next, stop은 overide 하지않는다 !!!!    
next 에서 정해진 조건에 도달했을 때 주문 방법 buy, sell, close, cancel 메소드 이용    
buy sell 은 order를 리턴    

* data (default: None) : 어떤 데이터에서 생성되었는가? (World에 여러 데이터가 있는 경우)    
* size : 주문량  
* price : 가격  
* exectype : 조건부 발동 (Order.Market: 시장가, Order.Limit: 다음틱 등)  
* plimit, tradeid, ..  

Order Notification은 notify_order, notify_trade, notify_cashvalue 등의 메소드를 이용합니다.   
메소드를 overide하여 로그를 남기거나 Slack 혹은 Telegram 과 같은 메신저를 연결해 모바일로 푸쉬 알림을 발생시킬 수도 있습니다.   
각 notification은 next가 한번 실행될 때 마다 함께 실행됩니다.

In [15]:
from datetime import datetime
import backtrader as bt

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

data = bt.feeds.GenericCSVData(
            dataname="kospi200_raw.csv",

            fromdate=datetime(2018, 1, 2),
            todate=datetime(2022, 12, 29),

            nullvalue=0.0,

            dtformat=('%Y-%m-%d'),
            #tmformat=('%I-%p'),

            datetime=0,
            time=-1,
            high=3,
            low=4,
            open=2,
            close=1,
            volume=5,
            openinterest=-1,

                        headers = True,
                        separator = ','
        )
cerebro.adddata(data)

cerebro.broker.setcash(10000000) # 초기 투자 자금 설정 : 천만원
cerebro.broker.setcommission(commission=0.0014) 
cerebro.run()
cerebro.plot()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xeb in position 1: illegal multibyte sequence

In [37]:
import backtrader as bt
import pandas as pd
import datetime

class DaillyBuy(bt.Strategy):
    def __init__(self):
        pass
    def next(self):
        self.buy(size=10)

In [38]:
import backtrader.feeds as btfeeds

class customCSV(btfeeds.GenericCSVData):
    params=(
        ('dtformat', '%Y-%m-%d'),
        ('datetime', 0),
        ('time', -1),
        ('open', 2),
        ('high', 3),
        ('low', 4),
        ('close', 1),
        ('volume', 5),
        ('openinterest', -1),
        )

In [39]:
data = customCSV(dataname='005930.csv')

In [40]:
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.broker.setcash(1000000)
cerebro.addstrategy(DaillyBuy)

0

In [41]:
back_init = cerebro.broker.getvalue()

In [42]:
back_init

1000000

In [43]:
cerebro.run()

In [44]:
result=cerebro.broker.getvalue()

In [45]:
result = cerebro.broker.getvalue()

In [18]:
df.columnsa

Index(['date', 'close', 'open', 'high', 'low', 'volume', 'name'], dtype='object')

In [33]:
with open('kospi200_raw.csv','r',encoding='utf-8') as f :

SyntaxError: incomplete input (3766483592.py, line 1)

In [36]:
import FinanceDataReader as fdr
df = fdr.DataReader('005930','2019-01-02','2020-07-31')
df.to_csv('005930.csv')